# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#498 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846853282190                   -0.70    4.5
  2   -7.852325590383       -2.26       -1.53    1.0
  3   -7.852646035763       -3.49       -2.52    3.0
  4   -7.852646679415       -6.19       -3.34    2.8
  5   -7.852646686179       -8.17       -4.87    1.0
  6   -7.852646686726       -9.26       -5.28    4.2
  7   -7.852646686730      -11.40       -6.07    2.0
  8   -7.852646686730      -12.84       -7.52    1.5
  9   -7.852646686730      -15.05       -7.88    2.0
 10   -7.852646686730      -14.75       -8.69    1.0
 11   -7.852646686730   +    -Inf       -9.62    3.2
 12   -7.852646686730   +  -15.05       -8.35    2.0
 13   -7.852646686730      -14.75       -8.59    1.8
 14   -7.852646686730   +  -15.05       -8.06    2.8
 15   -7.852646686730      -15.05       -8.00    1.8
 16   -7.852646686730   +  -14.75       -8.83    2.8
 17   -7.852646686730   +  -14.75       -9.82 

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846845619643                   -0.70           4.0
  2   -7.852527154431       -2.25       -1.62   0.80    5.0
  3   -7.852610727966       -4.08       -2.79   0.80    5.2
  4   -7.852646565064       -4.45       -3.57   0.80    7.2
  5   -7.852646683574       -6.93       -4.53   0.80    9.0
  6   -7.852646686679       -8.51       -5.10   0.80   10.5
  7   -7.852646686725      -10.34       -6.15   0.80   11.5
  8   -7.852646686730      -11.29       -7.17   0.80   13.2
  9   -7.852646686730      -13.37       -7.66   0.80   15.5
 10   -7.852646686730      -15.05       -9.42   0.80   15.8
 11   -7.852646686730   +  -15.05       -9.29   0.80   19.0
 12   -7.852646686730      -15.05      -10.17   0.80   18.8
 13   -7.852646686730   +  -15.05      -11.39   0.80   21.2
 14   -7.852646686730   +  -14.75      -11.95   0.80   23.2
 15   -7.852646686730      -14.45      -

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.355381e+01     3.376970e+00
 * time: 0.4185318946838379
     1     8.849328e-01     1.959631e+00
 * time: 0.6492960453033447
     2    -1.386951e+00     1.967992e+00
 * time: 0.6727890968322754
     3    -3.451027e+00     1.696164e+00
 * time: 0.7061259746551514
     4    -4.535229e+00     1.658271e+00
 * time: 0.7395029067993164
     5    -6.362141e+00     1.224468e+00
 * time: 0.7729969024658203
     6    -7.198222e+00     8.405671e-01
 * time: 0.8063139915466309
     7    -7.444800e+00     7.497344e-01
 * time: 0.8295550346374512
     8    -7.656511e+00     2.406752e-01
 * time: 0.8527300357818604
     9    -7.718685e+00     1.080332e-01
 * time: 0.8760290145874023
    10    -7.763465e+00     1.308765e-01
 * time: 0.8989279270172119
    11    -7.791415e+00     9.702478e-02
 * time: 0.9215579032897949
    12    -7.816125e+00     8.366262e-02
 * time: 0.9451370239257812
    13    -7.836388e+00     5.304275e-02
 * time: 0.9682779312

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846821415469                   -0.70    4.2
  2   -7.852326778993       -2.26       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686707                   -2.54
  2   -7.852646686730      -10.65       -5.89
  3   -7.852646686730   +  -15.05      -12.58
|ρ_newton - ρ_scf|  = 1.5792200281446307e-13
|ρ_newton - ρ_scfv| = 5.580680247674236e-14
|ρ_newton - ρ_dm|   = 2.582318257471268e-10
